In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from scripts.models import Building, Tag, RealestateDeal

BUILDING_AGE_THRESHOLD = 5

engine = create_engine(os.getenv("DATABASE_URL"), echo=False)

Session = sessionmaker(bind=engine)
session = Session()


def get_price(price):
    if price == "1억 이하":
        return (0, 10000)
    elif price == "1~3억":
        return (10001, 30000)
    elif price == "3~5억":
        return (30001, 50000)
    elif price == "5~10억":
        return (50001, 100000)
    elif price == "10억 이상":
        return (100001, None)


def get_floor(floor):
    if floor == "전체":
        return None
    elif floor == "1~5층 (저층)":
        return (1, 5)
    elif floor == "6~8층 (중층)":
        return (6, 8)
    elif floor == "9층 이상 (고층)":
        return (9, None)

In [8]:
from datetime import datetime

latest_deal_subquery = (
    session.query(
        RealestateDeal.building_id,
        func.max(
            RealestateDeal.contract_year * 10000
            + RealestateDeal.contract_month * 100
            + RealestateDeal.contract_day
        ).label("max_date"),
    )
    .group_by(RealestateDeal.building_id)
    .subquery()
)

query = session.query(Building).join(RealestateDeal)

query = query.join(
    latest_deal_subquery,
    (RealestateDeal.building_id == latest_deal_subquery.c.building_id)
    & (
        RealestateDeal.contract_year * 10000
        + RealestateDeal.contract_month * 100
        + RealestateDeal.contract_day
        == latest_deal_subquery.c.max_date
    ),
)

new_building = True
building_type = "전체"
tags = []
size = (10, 80)
price_range = get_price("1~3억")
floor = get_floor("9층 이상 (고층)")


if tags:
    for tag in tags:
        query = query.filter(
            session.query(Tag)
            .filter(Tag.building_id == Building.id, Tag.label == tag)
            .exists()
        )


if new_building:
    query = query.filter(
        Building.construction_year > datetime.now().year - BUILDING_AGE_THRESHOLD
    )


if building_type and building_type != "전체":
    query = query.filter(Building.purpose == building_type)


query = query.filter(Building.area_sqm.between(size[0], size[1]))


if price_range[1] is None:
    query = query.filter(RealestateDeal.transaction_price_million >= price_range[0])
else:
    query = query.filter(
        RealestateDeal.transaction_price_million.between(price_range[0], price_range[1])
    )


query = query.group_by(Building.id)


if floor:
    if floor[1] is None:
        query = query.filter(Building.floor >= floor[0])
    else:
        query = query.filter(Building.floor.between(floor[0], floor[1]))


buildings = query.all()
for building in buildings:
    print(building)

<Building(address_id=37096, name=갑을명가시티1, construction_year=2025, purpose=오피스텔, area_sqm=19.73, floor=18)>
<Building(address_id=5013, name=에스아이팰리스송파올림픽공원, construction_year=2021, purpose=아파트, area_sqm=16.60, floor=10)>
<Building(address_id=40953, name=동대문 푸르지오시티, construction_year=2025, purpose=오피스텔, area_sqm=23.47, floor=19)>
<Building(address_id=34682, name=이문휘경 지웰에스테이트, construction_year=2021, purpose=오피스텔, area_sqm=17.45, floor=18)>
<Building(address_id=20515, name=더이음, construction_year=2021, purpose=오피스텔, area_sqm=17.75, floor=15)>
<Building(address_id=29545, name=Tb view, construction_year=2024, purpose=오피스텔, area_sqm=29.25, floor=10)>
<Building(address_id=38443, name=구로 오네뜨 시티, construction_year=2022, purpose=오피스텔, area_sqm=21.39, floor=9)>
<Building(address_id=28521, name=한원힐트리움_NK, construction_year=2022, purpose=오피스텔, area_sqm=33.22, floor=9)>
<Building(address_id=36721, name=동우리즈힐스, construction_year=2022, purpose=오피스텔, area_sqm=16.67, floor=13)>
<Building(address_id=18347,